In [1]:
!pip install pycocotools -q
!pip install scikit-multilearn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
import json
import shutil
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from google.colab import drive
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/Dental Dissertation/before_filtered_opg_dataset_for_LLM3.zip'
base_dir = './extracted_opg/merged_opg_dataset'
shutil.rmtree(base_dir, ignore_errors=True)
os.makedirs(base_dir, exist_ok=True)
!unzip -o "{zip_path}" -d "{base_dir}"

Mounted at /content/drive
Archive:  /content/drive/MyDrive/Dental Dissertation/before_filtered_opg_dataset_for_LLM3.zip
   creating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/
   creating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/test/
  inflating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/test/output_1-2-392-200036-9125-4-0-554652783-21813480-3503410942_cropped_jpg.rf.4eee8a846f877c061330dcda6fc6c51c.jpg  
  inflating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/test/output_1-2-392-200036-9125-4-0-537861687-12769512-3503410942_cropped_jpg.rf.e3a162a5dab2565937b83c8c3767546a.jpg  
  inflating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/test/output_1-2-392-200036-9125-4-0-554643314-194894056-3503410942_cropped_jpg.rf.046e008b2a06d717d2d54734409e9cdd.jpg  
  inflating: ./extracted_opg/merged_opg_dataset/merged_opg_dataset/test/output_1-2-392-200036-9125-4-0-554678816-476502248-3503410942_cropped_jpg.rf.1334812438c956fe4a62c395210c

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt

annotation_path = '/content/extracted_opg/merged_opg_dataset/merged_opg_dataset/train/_annotations.coco.json'
images_path = '/content/extracted_opg/merged_opg_dataset/train/'

coco = COCO(annotation_path)
image_ids = coco.getImgIds()
# Get all categories
categories = coco.loadCats(coco.getCatIds())

print("Available categories in the COCO annotation file:")
for category in categories:
    print(f"  - ID: {category['id']}, Name: {category['name']}")

# Get all image IDs
image_ids = coco.getImgIds()

# Check if image_ids is empty and handle it
if not image_ids:
    print("No images found in the annotation file. Exiting...")
    exit()

# Get image info
image_info = coco.loadImgs(image_ids[0])[0] # Get the first image
print("\nExample image information:")
print(json.dumps(image_info, indent=2))


#Get annotation info for the first image
annotation_ids = coco.getAnnIds(imgIds=image_ids[0])
annotations = coco.loadAnns(annotation_ids)
print("\nExample annotation information:")
print(json.dumps(annotations, indent=2))

loading annotations into memory...
Done (t=0.91s)
creating index...
index created!
Available categories in the COCO annotation file:
  - ID: 0, Name: OPG-data
  - ID: 1, Name: 11
  - ID: 2, Name: 12
  - ID: 3, Name: 13
  - ID: 4, Name: 14
  - ID: 5, Name: 15
  - ID: 6, Name: 16
  - ID: 7, Name: 17
  - ID: 8, Name: 18
  - ID: 9, Name: 21
  - ID: 10, Name: 22
  - ID: 11, Name: 23
  - ID: 12, Name: 24
  - ID: 13, Name: 25
  - ID: 14, Name: 26
  - ID: 15, Name: 27
  - ID: 16, Name: 28
  - ID: 17, Name: 31
  - ID: 18, Name: 32
  - ID: 19, Name: 33
  - ID: 20, Name: 34
  - ID: 21, Name: 35
  - ID: 22, Name: 36
  - ID: 23, Name: 37
  - ID: 24, Name: 38
  - ID: 25, Name: 41
  - ID: 26, Name: 42
  - ID: 27, Name: 43
  - ID: 28, Name: 44
  - ID: 29, Name: 45
  - ID: 30, Name: 46
  - ID: 31, Name: 47
  - ID: 32, Name: 48
  - ID: 33, Name: RC-treated
  - ID: 34, Name: RC-treated-and-root-stump
  - ID: 35, Name: RPD clasp
  - ID: 36, Name: bridge
  - ID: 37, Name: calculus
  - ID: 38, Name: cantile

Filtering and Grouping Logic

In [ ]:
original_id_to_name = {
    0: 'opg-data', 1: '11', 2: '12', 3: '13', 4: '14', 5: '15', 6: '16', 7: '17', 8: '18',
    9: '21', 10: '22', 11: '23', 12: '24', 13: '25', 14: '26', 15: '27', 16: '28', 17: '31',
    18: '32', 19: '33', 20: '34', 21: '35', 22: '36', 23: '37', 24: '38', 25: '41', 26: '42',
    27: '43', 28: '44', 29: '45', 30: '46', 31: '47', 32: '48',
    33: 'rc-treated',
    34: 'rc-treated-and-root-stump',
    35: 'rpd clasp',
    36: 'bridge',
    37: 'calculus',
    38: 'cantilever',
    39: 'caries',
    40: 'cast partial denture',
    41: 'condensing osteitis',
    42: 'crown',
    43: 'cyst',
    44: 'distomolar',
    45: 'grossly-decayed',
    46: 'hemimandibulectomy',
    47: 'impacted',
    48: 'implant',
    49: 'inter radicularradiolucency',
    50: 'mini plates',
    51: 'missing',
    52: 'peri implant bone loss',
    53: 'periapical radiolucency',
    54: 'radiolucency',
    55: 'restoration',
    56: 'retained decidious',
    57: 'root-stump',
    58: 'supernumerary',
    59: 'tooth-bud',
    60: 'vascular clips',
    61: 'wire splint',
    62: 'joint crowns'
}

# Define Grouping Map (Original Name -> New Grouped Name)
group_map = {
    # Caries Group
    'caries': 'caries',
    'grossly-decayed': 'caries',

    # Restoration Group
    'restoration': 'restoration',
    'bridge': 'restoration',
    'cantilever': 'restoration',

    # Other preserved classes
    'calculus': 'calculus',
    'periapical radiolucency': 'periapical radiolucency',
    #'cyst': 'cyst',
    'impacted': 'impacted',
    'root-stump': 'root-stump',
    'crown': 'crown',
    'joint crowns': 'crown',
    'implant': 'implant',
    'rc-treated': 'rc-treated'
}

# Add Teeth IDs
print("Adding teeth IDs (1-32) to preserved classes...")
for i in range(1, 33):
    tooth_name = original_id_to_name.get(i)
    if tooth_name:
        group_map[tooth_name] = tooth_name

# Create Final Category List
final_categories_list = sorted(list(set(group_map.values())))
final_cat_to_id = {name: i for i, name in enumerate(final_categories_list)}
new_coco_categories = [{'id': i, 'name': name} for i, name in enumerate(final_categories_list)]

print(f"Original categories found: {len(original_id_to_name)}")
print(f"New categories to be created ({len(final_categories_list)} total):")
for cat_info in new_coco_categories:
    print(f"  - ID: {cat_info['id']}, Name: {cat_info['name']}")

Adding teeth IDs (1-32) to preserved classes...
Original categories found: 63
New categories to be created (41 total):
  - ID: 0, Name: 11
  - ID: 1, Name: 12
  - ID: 2, Name: 13
  - ID: 3, Name: 14
  - ID: 4, Name: 15
  - ID: 5, Name: 16
  - ID: 6, Name: 17
  - ID: 7, Name: 18
  - ID: 8, Name: 21
  - ID: 9, Name: 22
  - ID: 10, Name: 23
  - ID: 11, Name: 24
  - ID: 12, Name: 25
  - ID: 13, Name: 26
  - ID: 14, Name: 27
  - ID: 15, Name: 28
  - ID: 16, Name: 31
  - ID: 17, Name: 32
  - ID: 18, Name: 33
  - ID: 19, Name: 34
  - ID: 20, Name: 35
  - ID: 21, Name: 36
  - ID: 22, Name: 37
  - ID: 23, Name: 38
  - ID: 24, Name: 41
  - ID: 25, Name: 42
  - ID: 26, Name: 43
  - ID: 27, Name: 44
  - ID: 28, Name: 45
  - ID: 29, Name: 46
  - ID: 30, Name: 47
  - ID: 31, Name: 48
  - ID: 32, Name: calculus
  - ID: 33, Name: caries
  - ID: 34, Name: crown
  - ID: 35, Name: impacted
  - ID: 36, Name: implant
  - ID: 37, Name: periapical radiolucency
  - ID: 38, Name: rc-treated
  - ID: 39, Name: r

Load, Filter, and Combine All Data

In [6]:
base_dir = './extracted_opg/merged_opg_dataset/merged_opg_dataset'

In [ ]:
all_images_map = {} # Key: original_path, Value: img_info
all_annotations_map = defaultdict(list) # Key: original_path, Value: list of new_anns

splits = ['train', 'valid', 'test']
print("\nLoading and filtering data from all splits...")

original_total_images = 0

for split in splits:
    split_path = os.path.join(base_dir, split)
    ann_file_path = os.path.join(split_path, '_annotations.coco.json')

    if not os.path.exists(ann_file_path):
        print(f"Warning: Annotation file not found at {ann_file_path}")
        continue

    with open(ann_file_path, 'r') as f:
        data = json.load(f)

    split_image_count = len(data['images'])
    print(f"Found {split_image_count} images in original '{split}' split.")
    original_total_images += split_image_count

    img_id_to_info = {img['id']: img for img in data['images']}

    # Find all annotations to keep and group them by image
    temp_anns_for_image = defaultdict(list)
    for ann in data['annotations']:
        original_cat_id = ann['category_id']
        original_cat_name = original_id_to_name.get(original_cat_id)

        # Check if this category is one want to keep
        if original_cat_name and original_cat_name in group_map:
            new_cat_name = group_map[original_cat_name]
            new_cat_id = final_cat_to_id[new_cat_name]

            # Create a new annotation with the updated category ID
            new_ann = ann.copy()
            new_ann['category_id'] = new_cat_id
            temp_anns_for_image[ann['image_id']].append(new_ann)

    # Add images and their filtered annotations to the global maps
    for img_id, anns in temp_anns_for_image.items():
        if img_id in img_id_to_info:
            img_info = img_id_to_info[img_id]
            original_path = os.path.join(split_path, img_info['file_name'])

            if not os.path.exists(original_path):
                print(f"Warning: Image file not found at {original_path}")
                continue

            # Use original_path as a unique key for images
            if original_path not in all_images_map:
                all_images_map[original_path] = img_info

            all_annotations_map[original_path].extend(anns)

print("\n--- Summary ---")
print(f"Original total images (across all splits): {original_total_images}")
print(f"Filtered images (with desired annotations): {len(all_images_map)}")


Loading and filtering data from all splits...
Found 1873 images in original 'train' split.
Found 443 images in original 'valid' split.
Found 221 images in original 'test' split.

--- Summary ---
Original total images (across all splits): 2537
Filtered images (with desired annotations): 2514


Perform Stratified Split (70/15/15)

In [ ]:
print("Performing 70/15/15 stratified split...")

image_paths = list(all_images_map.keys())
num_images = len(image_paths)
num_classes = len(final_categories_list)

# Build X (image paths) and Y (multi-label binary matrix)
X = np.array(image_paths).reshape(-1, 1)
Y = np.zeros((num_images, num_classes))

for i, path in enumerate(image_paths):
    anns = all_annotations_map[path]
    for ann in anns:
        new_cat_id = ann['category_id']
        Y[i, new_cat_id] = 1

# 70/15/15 split
# Split into 70% train and 30% temp (valid + test)
try:
    X_train, Y_train, X_temp, Y_temp = iterative_train_test_split(X, Y, test_size = 0.3)

    # Split the 30% temp 50/50 to get 15% valid and 15% test
    X_val, Y_val, X_test, Y_test = iterative_train_test_split(X_temp, Y_temp, test_size = 0.5)
    train_paths = X_train.flatten().tolist()
    val_paths = X_val.flatten().tolist()
    test_paths = X_test.flatten().tolist()

    split_map = {'train': train_paths, 'valid': val_paths, 'test': test_paths}

    print("Split successful:")
    print(f"  Train: {len(train_paths)} images ({len(train_paths)/num_images:.1%})")
    print(f"  Valid: {len(val_paths)} images ({len(val_paths)/num_images:.1%})")
    print(f"  Test:  {len(test_paths)} images ({len(test_paths)/num_images:.1%})")

except ValueError as e:
    print(f"\n--- Error during stratified split ---")
    print(f"Error: {e}")

    # Fallback to standard random split
    X_train_val, X_test = train_test_split(X, test_size=0.15, random_state=42)
    X_train, X_val = train_test_split(X_train_val, test_size=0.1765, random_state=42) # 0.15 / 0.85

    train_paths = X_train.flatten().tolist()
    val_paths = X_val.flatten().tolist()
    test_paths = X_test.flatten().tolist()

    split_map = {'train': train_paths, 'valid': val_paths, 'test': test_paths}

Performing 70/15/15 stratified split...
Split successful:
  Train: 1763 images (70.1%)
  Valid: 378 images (15.0%)
  Test:  373 images (14.8%)


Create New Dataset Files

In [ ]:
new_base_dir = './filtered_opg_dataset_for_LLM'
shutil.rmtree(new_base_dir, ignore_errors=True)

print(f"\nCreating new dataset at {new_base_dir}...")

all_counts = defaultdict(lambda: Counter())

for split_name, paths in split_map.items():
    new_split_dir = os.path.join(new_base_dir, split_name)
    os.makedirs(new_split_dir, exist_ok=True)

    # New COCO structure for split
    new_coco_data = {
        'info': {'description': f'Filtered OPG Dataset - {split_name} split'},
        'licenses': [],
        'categories': new_coco_categories,
        'images': [],
        'annotations': []
    }

    current_new_img_id = 0
    current_new_ann_id = 0

    for old_path in paths:
        img_info = all_images_map[old_path]

        # Add new image info to COCO data
        new_img_info = img_info.copy()
        new_img_info['id'] = current_new_img_id
        new_coco_data['images'].append(new_img_info)

        # Copy image file to new directory
        shutil.copy(old_path, os.path.join(new_split_dir, img_info['file_name']))

        # Get and add new annotations
        anns = all_annotations_map[old_path]
        for ann in anns:
            new_ann = ann.copy()
            new_ann['id'] = current_new_ann_id
            new_ann['image_id'] = current_new_img_id
            new_coco_data['annotations'].append(new_ann)

            # Count for final report
            cat_name = final_categories_list[new_ann['category_id']]
            all_counts[split_name][cat_name] += 1

            current_new_ann_id += 1

        current_new_img_id += 1

    # Save new annotation file
    new_ann_file_path = os.path.join(new_split_dir, '_annotations.coco.json')
    with open(new_ann_file_path, 'w') as f:
        json.dump(new_coco_data, f, indent=2)

    print(f"Created {split_name} split with {current_new_img_id} images and {current_new_ann_id} annotations.")

print("\nDataset filtering and splitting complete!")

# Creating new dataset at ./filtered_opg_dataset_for_LLM...
# Created train split with 1540 images and 54614 annotations.
# Created valid split with 338 images and 11925 annotations.
# Created test split with 335 images and 11980 annotations.

# Creating new dataset at ./filtered_opg_dataset_for_LLM...
# Created train split with 1628 images and 55012 annotations.
# Created valid split with 342 images and 12174 annotations.
# Created test split with 346 images and 11925 annotations.

# Creating new dataset at ./filtered_opg_dataset_for_LLM...
# Created train split with 1696 images and 55463 annotations.
# Created valid split with 360 images and 12123 annotations.
# Created test split with 360 images and 12104 annotations.

# Creating new dataset at ./filtered_opg_dataset_for_LLM...
# Created train split with 1763 images and 56041 annotations.
# Created valid split with 378 images and 12433 annotations.
# Created test split with 373 images and 12036 annotations


Creating new dataset at ./filtered_opg_dataset_for_LLM...
Created train split with 1763 images and 56041 annotations.
Created valid split with 378 images and 12433 annotations.
Created test split with 373 images and 12036 annotations.

Dataset filtering and splitting complete!


Display Final Class Counts

In [ ]:
print("\n--- Annotation Counts per Class in New Splits ---")

# Create a DataFrame from the counts dictionary
df = pd.DataFrame(all_counts).fillna(0).astype(int)
df = df.reindex(columns=['train', 'valid', 'test'], fill_value=0)

df['Total'] = df.sum(axis=1)
df.loc['Total'] = df.sum()
df = df.sort_values(by='Total', ascending=False)

print(df.to_markdown(numalign="left", stralign="left"))

print("\n--- Image Counts in New Splits ---")
print(f"Train: {len(split_map['train'])} images")
print(f"Valid: {len(split_map['valid'])} images")
print(f"Test:  {len(split_map['test'])} images")


--- Annotation Counts per Class in New Splits ---
|                         | train   | valid   | test   | Total   |
|:------------------------|:--------|:--------|:-------|:--------|
| Total                   | 56041   | 12433   | 12036  | 80510   |
| caries                  | 4208    | 906     | 861    | 5975    |
| calculus                | 2344    | 594     | 544    | 3482    |
| rc-treated              | 1934    | 459     | 418    | 2811    |
| restoration             | 1891    | 398     | 368    | 2657    |
| impacted                | 1606    | 332     | 318    | 2256    |
| 43                      | 1493    | 326     | 325    | 2144    |
| 33                      | 1480    | 328     | 323    | 2131    |
| 42                      | 1470    | 319     | 316    | 2105    |
| 13                      | 1457    | 321     | 314    | 2092    |
| 32                      | 1451    | 321     | 315    | 2087    |
| 41                      | 1440    | 316     | 312    | 2068    |
| 23       

In [ ]:
import time
folder_to_zip = './filtered_opg_dataset_for_LLM'
zip_file_name = 'filtered_opg_dataset_for_LLM.zip'
drive_destination_path = f'/content/drive/MyDrive/Dental Dissertation/{zip_file_name}'

print(f"Zipping the folder: {folder_to_zip}")
start_time = time.time()

!zip -r -q {zip_file_name} {folder_to_zip}

zip_time = time.time()
print(f"Zipping complete in {zip_time - start_time:.2f} seconds.")
print(f"Copying {zip_file_name} to your Google Drive...")
!cp {zip_file_name} {drive_destination_path}

copy_time = time.time()
print(f"Copying complete in {copy_time - zip_time:.2f} seconds.")

print("\n Success!")
print(f"New dataset is saved as a zip file at: {drive_destination_path}")

Attempting to zip the folder: ./filtered_opg_dataset_for_LLM
Zipping complete in 43.25 seconds.
Copying filtered_opg_dataset_for_LLM.zip to your Google Drive...
cp: target 'Dissertation/filtered_opg_dataset_for_LLM.zip' is not a directory
Copying complete in 0.11 seconds.

✅ Success!
Your new dataset is saved as a zip file at: /content/drive/MyDrive/Dental Dissertation/filtered_opg_dataset_for_LLM.zip


In [ ]:
from google.colab import files
zip_file_name = 'filtered_opg_dataset_for_LLM.zip'

print(f"Preparing to download {zip_file_name}...")
files.download(zip_file_name)

Preparing to download filtered_opg_dataset_for_LLM.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>